# NOT FINISHED!!! - Parameter recocery

This notebook is a test for model fitting. Here we use agents to simulate the task, then we try to recover parameters by fitting the model to the simulated data. Model fitting is through Likelihood estimation.

This notebook can also test model comparison tools.

---
# Setup

In [ ]:
# @title Imports
import gymnasium as gym
from gymnasium.wrappers import RecordEpisodeStatistics
import gymnasium as gym
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

from popy.simulation_tools import *

from simulation_helpers import simulate_agent, estimate_ll

In [2]:
# @title Helpers
...

In [3]:
# @title Plotting functions
...

# Section 1: Simulate data

In [4]:
# Create the environment
env = gym.make("zsombi/monkey-bandit-task-v0", n_arms=3, max_episode_steps=1_000)

# Apply the custom reward wrapper
env = ChangeZeroRewardToNegativeOne(env)

# initialize the Q-learner agent
agent = QLearner(env=env)

params_true = {
    # initialize the Q-learner agent
    'alpha': .1,
    'beta': 100,  # rate of exploration (i.e. random actions)
    'structure_aware': False
}

for k, v in params_true.items():
    setattr(agent, k, v)


In [5]:
# simulate the agent
behavior = simulate_agent(agent)

print(f'p(best action): {np.mean(behavior["action"] == behavior["best_arm"]):.2f}')

# plot the behavior
behavior

p(best action): 0.60


,action,reward,best_arm
0,0,1,1
1,0,-1,1
2,2,-1,1
3,1,1,1
4,1,1,1
...,...,...,...
995,0,-1,2
996,0,-1,2
997,0,-1,2
998,0,-1,2


# Section 2: Recover parameters

In this section we fit the free parameters of the model and try to recover the true parameters.

In [6]:
# initialize a naive Q-learner agent
agent = QLearner(env=env, structure_aware=True)
params_0 = [0.3, 5]

result = scipy.optimize.minimize(estimate_ll, params_0, args=(agent, behavior), method="Nelder-Mead") #, options={'maxiter': 1_000})

print(result)
print("")
print(f"MLE: alpha = {result.x[0]:.2f} (true value = {params_true['alpha']})")
print(f"MLE: beta = {result.x[1]:.2f} (true value = {params_true['beta']})")

       message: Maximum number of function evaluations has been exceeded.
       success: False
        status: 1
           fun: 390.2987798316901
             x: [-2.286e-02  1.063e+00]
           nit: 150
          nfev: 400
 final_simplex: (array([[-2.286e-02,  1.063e+00],
                       [-2.286e-02,  1.063e+00],
                       [-2.286e-02,  1.063e+00]]), array([ 3.903e+02,  6.502e+02,  6.502e+02]))

MLE: alpha = -0.02 (true value = 0.1)
MLE: beta = 1.06 (true value = 100)


# Section 3: Model comparison

In [7]:
...

Ellipsis